# View of Exposures Flats


- work with Weakly_2023_01
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/03/27



In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

In [ ]:
import os
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Butler
import lsst.daf.butler as dafButler

In [ ]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

## List of Exposures from raw

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program','jd','mjd'])

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 2:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

In [ ]:
df_exposure

## Selection of flats raw-exposures

In [ ]:
df_flat = df_exposure[df_exposure.type == 'flat']

In [ ]:
df_flat = df_flat.sort_values(by="day_obs",ascending=True)

In [ ]:
df_flat.tail(50)

# List of dates

In [ ]:
list_of_dates_flats = df_flat['day_obs'].unique() 
list_of_dates_flats

# List of filters

In [ ]:
list_of_filters = df_flat['filter'].unique()
list_of_filters

# Selection of the filter

In [ ]:
cut_filter = df_flat['filter'] == 'empty~empty'

In [ ]:
for the_date in list_of_dates_flats:
    cut_date = df_flat['day_obs'] == the_date
    cut_combined = cut_date & cut_filter
    
    df_flat_this_date = df_flat[cut_combined]
    
    N = len(df_flat_this_date) 
    if N >0:
        print("=======================================================================")   
        print(f"{the_date} :: N={N}")
        print(df_flat_this_date.id.values)
    

# View

In [ ]:
cut_date = df_flat['day_obs'] == 20230214
cut_combined = cut_date & cut_filter

In [ ]:
df_flat_this_date = df_flat[cut_combined]

In [ ]:
N = len(df_flat_this_date)
df_flat_this_date = df_flat_this_date.sort_values(by="id").reset_index()

In [ ]:
df_flat_this_date 

In [ ]:
collection='LATISS/raw/all'

# loop on exposures
for idx in range(N):
    exposure_selected=df_flat_this_date.iloc[idx].id 
    
   
    raw_img = butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)

    fig = plt.figure(figsize=(12,10))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('linear', 'zscale',None)
    title = f"raw flat : {exposure_selected}"
    afw_display.mtv(raw_img.image,title=title)
    #plt.gca().axis('off')
    

## Assembly task
- examples here : 
https://github.com/lsst/ip_isr/blob/main/tests/test_assembleCcd.py

In [ ]:
ass_config = AssembleCcdConfig(doTrim=True,keysToRemove=['SHEEP', 'MONKEYS', 'ZSHEEP'])
ass_task = AssembleCcdTask(config=ass_config)

In [ ]:
collection='LATISS/raw/all'

# loop on exposures
for idx in range(N):
    exposure_selected=df_flat_this_date.iloc[idx].id 
    
   
    raw_img = butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    ass_img = ass_task.assembleCcd(raw_img)

    fig = plt.figure(figsize=(12,10))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('linear', 'zscale',None)
    title = f"raw flat : {exposure_selected}"
    afw_display.mtv(ass_img.image,title=title)
    #plt.gca().axis('off')
    

# Select One Flat

https://github.com/lsst/cp_pipe/blob/main/python/lsst/cp/pipe/cpFlatNormTask.py

In [ ]:
idx_sel= 0
exposure_selected=df_flat_this_date.iloc[idx_sel].id     
raw_img = butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
ass_img = ass_task.assembleCcd(raw_img)

In [ ]:
for ampIdx, amp in enumerate(ass_img.getDetector()):
    ampName = amp.getName()
    # This can work only on postISRCCD
    ampExp = ass_img.Factory(ass_img, amp.getBBox())
    
    fig = plt.figure(figsize=(5,10))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('linear', 'zscale',None)
    title = f"raw flat : {exposure_selected}, ampl = {ampName}"
    afw_display.mtv(ampExp.image,title=title)

# Perform ISR on the Flat Exposure

- overscan
- defect
- bias

In [ ]:
for c in sorted(registry.queryCollections()):
    if "calib" in c:
        print(c)

In [ ]:
# configuration
isr_config =  IsrTaskConfig()

In [ ]:
isr_config.doDark = False
isr_config.doFlat =  False
isr_config.doFringe = False
isr_config.doDefect = True
isr_config.doLinearize = False
isr_config.doCrosstalk =  False
isr_config.doSaturationInterpolation = False
isr_config.overscan.fitType: 'MEDIAN_PER_ROW'
isr_config.doBias: True

In [ ]:
isr_task = IsrTask(config=isr_config)

In [ ]:
calibType = 'bias'
physical_filter = 'empty~empty'
cameraName = 'LATISS'
# Collection name containing the verification outputs.
calibCollections = ['LATISS/calib','LATISS/raw/all',\
'LATISS/calib/DM-28636',\
'LATISS/calib/DM-28636/curated/19700101T000000Z',\
'LATISS/calib/DM-28636/curated/20180101T000000Z',\
'LATISS/calib/DM-28636/unbounded',\
'LATISS/calib/DM-33875',\
'LATISS/calib/DM-36484/bias.20221005a',\
'LATISS/calib/DM-36484/biasGen.20221005a',\
'LATISS/calib/DM-36484/biasGen.20221005a/20221006T000101Z',\
'LATISS/calib/DM-36484/dark.20221006a',\
'LATISS/calib/DM-36484/darkGen.20221005a',\
'LATISS/calib/DM-36484/darkGen.20221005a/20221006T222501Z',\
'LATISS/calib/DM-36484/darkGen.20221006a',\
'LATISS/calib/DM-36484/darkGen.20221006a/20221006T222921Z',\
'LATISS/calib/DM-36484/flat-SDSSg.20221006a',\
'LATISS/calib/DM-36484/flat-SDSSi.20221006a',\
'LATISS/calib/DM-36484/flat-SDSSr.20221006a',\
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a',\
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a/20221007T002703Z',\
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a',\
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a/20221007T003732Z',\
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a',\
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a/20221007T004708Z',\
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a',\
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a/20221006T233657Z',\
'LATISS/calib/DM-36484/verifyBias.20221005a',\
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T000747Z',\
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T213237Z',\
'LATISS/calib/DM-36484/verifyBias.20221005b',\
'LATISS/calib/DM-36484/verifyBias.20221005b/20221019T205236Z',\
'LATISS/calib/DM-36484/verifyDark.20221006a',\
'LATISS/calib/DM-36484/verifyDark.20221006a/20221006T224403Z',\
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a',\
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a/20221007T003418Z',\
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a',\
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a/20221007T004423Z',\
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a',\
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a/20221006T234341Z',\
'LATISS/calib/DM-36719',\
'LATISS/calib/DM-36719/bias.20221107',\
'LATISS/calib/DM-36719/biasGen.20221107a',\
'LATISS/calib/DM-36719/biasGen.20221107a/20221107T205127Z',\
'LATISS/calib/DM-36719/biasGen.20221107b',\
'LATISS/calib/DM-36719/biasGen.20221107b/20221107T213306Z',\
'LATISS/calib/DM-36719/dark.20221107',\
'LATISS/calib/DM-36719/darkGen.20221107a',\
'LATISS/calib/DM-36719/darkGen.20221107a/20221107T223409Z',\
'LATISS/calib/DM-36719/flat-SDSSg.20221107',\
'LATISS/calib/DM-36719/flat-SDSSi.20221107',\
'LATISS/calib/DM-36719/flat-SDSSr.20221107',\
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a',\
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a/20221108T002737Z',\
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a',\
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a/20221108T005202Z',\
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a',\
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a/20221107T235401Z',\
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a',\
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a/20221108T180421Z',\
'LATISS/calib/DM-36719/verifyBias.20221107b',\
'LATISS/calib/DM-36719/verifyBias.20221107b/20221107T220410Z',\
'LATISS/calib/DM-36719/verifyDark.20221107a',\
'LATISS/calib/DM-36719/verifyDark.20221107a/20221107T232823Z',\
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a',\
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a/20221108T004225Z',\
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a',\
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T012110Z',\
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T014950Z',\
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a',\
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a/20221108T000940Z',\
'LATISS/calib/DM-37587/flat-BG40.20230113a',\
'LATISS/calib/DM-37587/flat-OG550.20230113a',\
'LATISS/calib/DM-37587/flat-SDSSr.20230113a',\
'LATISS/calib/DM-37587/flatGen-BG40.20230113a',\
'LATISS/calib/DM-37587/flatGen-BG40.20230113a/20230113T211456Z',\
'LATISS/calib/DM-37587/flatGen-OG550.20230113b',\
'LATISS/calib/DM-37587/flatGen-OG550.20230113b/20230113T213305Z',\
'LATISS/calib/DM-37587/flatGen-SDSSr.20230113c',\
'LATISS/calib/DM-37587/flatGen-SDSSr.20230113c/20230113T214159Z',\
'LATISS/calib/DM-37587/verifyFlat-BG40.20230113a',\
'LATISS/calib/DM-37587/verifyFlat-BG40.20230113a/20230113T212641Z',\
'LATISS/calib/DM-37587/verifyFlat-OG550.20230113b',\
'LATISS/calib/DM-37587/verifyFlat-OG550.20230113b/20230113T213804Z',\
'LATISS/calib/DM-37587/verifyFlat-SDSSr.20230113c',\
'LATISS/calib/DM-37587/verifyFlat-SDSSr.20230113c/20230113T220628Z',\
'LATISS/calib/unbounded',\
'LSSTCam/calib/DM-36442/bias.20221026a',\
'LSSTCam/calib/DM-36442/bias.20221026b',\
'LSSTCam/calib/DM-36442/dark.20221026a',\
'LSSTCam/calib/DM-36442/dark.20221026b',\
'LSSTCam/calib/DM-36442/defects.20221026a',\
'LSSTCam/calib/DM-36442/defects.20221026b',\
'LSSTCam/calib/DM-36442/flat.20221026a',\
'LSSTCam/calib/DM-36442/flat.20221026b',\
'LSSTCam/calib/DM-36442/linearity.20221026a',\
'LSSTCam/calib/DM-36442/linearity.20221026b',\
'LSSTComCam/calib',\
'LSSTComCam/calib/DM-28636',\
'LSSTComCam/calib/DM-28636/unbounded',\
'LSSTComCam/calib/DM-33657',\
'LSSTComCam/calib/unbounded',\
'u/calib/DM-32209-20211013a-felh',\
'u/calib/DM-32209-20211013a-g',\
'u/czw/DM-28920/calib/bias.20210720',\
'u/czw/DM-28920/calib/dark.20210720a',\
'u/czw/DM-28920/calib/defect.20210720a',\
'u/czw/DM-28920/calib/flat.20210720',\
'u/czw/DM-37811/parOStest.20230201a/calib',\
'u/czw/DM-37811/parOStest.20230201a/calib/bias.20230201a',\
'u/czw/DM-37811/parOStest.20230201a/calib/dark.20230201b',\
'u/czw/DM-37811/parOStest.20230201a/calib/flat-SDSSi.20230201c',\
'u/czw/DM-37811/parOStest.20230202a/calib',\
'u/czw/DM-37811/parOStest.20230202a/calib/bias.20230202a',\
'u/czw/DM-37811/parOStest.20230202a/calib/dark.20230202a',\
'u/czw/DM-37811/parOStest.20230202a/calib/flat-BG40.20230207a',\
'u/czw/DM-37811/parOStest.20230202a/calib/flat-OG550.20230207a',\
'u/czw/DM-37811/parOStest.20230202a/calib/flat-SDSSg.20230203a',\
'u/czw/DM-37811/parOStest.20230202a/calib/flat-SDSSi.20230202a',\
'u/czw/DM-37811/parOStest.20230202a/calib/flat-SDSSr.20230203a',\
'u/czw/DM-38434/linearized/calib',\
'u/czw/DM-38434/linearized/calib/bias.20230324a',\
'u/czw/DM-38434/linearized/calib/dark.20230324a',\
'u/czw/DM-38434/linearized/calib/flat-SDSSr.20230324a',\
'u/czw/DM-38434/linearized/calib/linearity.',\
'u/czw/calibX.20220608']

In [ ]:
butler = dafButler.Butler(repo, collections=calibCollections)
camera = butler.get('camera', instrument=cameraName)
bias = butler.get('bias',instrument=cameraName,detector=0)
defects = butler.get('defects',instrument=cameraName,detector=0)

In [ ]:
fig = plt.figure(figsize=(12,10))
afw_display = afwDisplay.Display(frame=fig)
afw_display.scale('linear', 'zscale',None)
title = f"Master bias "
afw_display.mtv(bias.image,title=title)

### Perform  ISR on one raw Flat exposure

In [ ]:
isr_img = isr_task.run(raw_img,bias=bias,defects=defects)

In [ ]:
fig = plt.figure(figsize=(12,10))
afw_display = afwDisplay.Display(frame=fig)
afw_display.scale('linear', 'zscale',None)
title = f"unbiased flat : {exposure_selected}"
afw_display.mtv(isr_img.exposure.image,title=title)

### View each individual exposures

In [ ]:
for ampIdx, amp in enumerate(isr_img.exposure.getDetector()):
    ampName = amp.getName()
    # This can work only on postISRCCD
    ampExp = isr_img.exposure.Factory(isr_img.exposure, amp.getBBox())
    
    fig = plt.figure(figsize=(5,10))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('linear', 'zscale',None)
    title = f"unbiased flat : {exposure_selected}, ampl = {ampName}"
    afw_display.mtv(ampExp.image,title=title)